In [ ]:
# Les installation
!pip install findspark

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
import findspark
findspark.init('/opt/spark-3.0.1-bin-hadoop3.2') # Localisation de Apache Spark en local

In [3]:
# Creation de SparkContext et de SparkSession
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('Words Count') \
                    .getOrCreate()


23/10/28 11:22:45 WARN Utils: Your hostname, josarthur-HP-ProBook-450-G8-Notebook-PC resolves to a loopback address: 127.0.1.1; using 10.3.7.83 instead (on interface enp2s0)
23/10/28 11:22:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/28 11:22:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc = spark.sparkContext # Creation du context de spark

In [5]:
data_source = "/home/jos-arthur/Desktop/Workshop/TP/data/data.txt"

In [6]:
words_rdd = sc.textFile(data_source)

In [7]:
words_rdd.take(100)

['',
 "                    WILLIAM SHAKESPEARE'S",
 '',
 '                       ROMEO & JULIET',
 '',
 '   ADAPTED FOR THE SCREEN BY CRAIG PEARCE AND BAZ LUHRMANN',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                                       FINAL SHOOTING SCRIPT',
 '',
 '                                             October 6, 1995',
 '',
 '',
 '',
 '',
 'EXT.  HIGHWAY.  AFTERNOON.',
 '',
 'A ribbon of freeway stretching into a blue and pink late',
 'afternoon sky. A huge dark sedan, windows tinted gold,',
 'headlights blazing, powers directly for us.',
 '',
 'CUT TO: A heavy, low-slung, pickup truck traveling toward',
 'the sedan.',
 '',
 'WIDE SHOT: Sky, freeway, the cars closing.',
 '',
 'TIGHT ON: The sedan.',
 '',
 'TIGHT ON: The pickup.',
 '',
 'Like thunderous, jousting opponents, the cars pass in a',
 'deafening cacophony of noise.',
 '',
 'INT.  TRUCK.  AFTERNOON.',
 '',
 'TIGHT ON: 

In [8]:
words_rdd.count()

6247

## DATA CLEANING

In [9]:
def lower_clean_str(x):
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

## MAPREDUCE

In [10]:
words_rdd = words_rdd.map(lower_clean_str)

In [11]:
words_rdd.take(40)

['',
 '                    william shakespeares',
 '',
 '                       romeo  juliet',
 '',
 '   adapted for the screen by craig pearce and baz luhrmann',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                                       final shooting script',
 '',
 '                                             october 6 1995',
 '',
 '',
 '',
 '',
 'ext  highway  afternoon']

In [12]:
words_rdd = words_rdd.flatMap(lambda x: x.split(' ')) # Application de la fonction flatMap

In [13]:
words_rdd.take(5)

['', '', '', '', '']

In [14]:
words_rdd = words_rdd.filter(lambda x:x!='') # Application du filtre pour enlever les espace entre les mots

In [15]:
words_rdd.take(4)

['william', 'shakespeares', 'romeo', 'juliet']

In [16]:
words_count = words_rdd.map(lambda  word:(word,1)) # Application de la fonction Map()

In [17]:
words_count.take(4)

[('william', 1), ('shakespeares', 1), ('romeo', 1), ('juliet', 1)]

In [18]:
words_count_RBK=words_count.reduceByKey(lambda x,y:(x+y)).sortByKey() # Application de la fonction Reduce()

In [19]:
words_count_RBK.take(50)  # Resultat après l'application de la fonction reduce sur la fonction map et flatMap

[('1995', 1),
 ('21', 1),
 ('6', 1),
 ('60', 2),
 ('9mm', 2),
 ('a', 563),
 ('abandoned', 1),
 ('able', 1),
 ('about', 3),
 ('above', 12),
 ('abra', 24),
 ('abras', 3),
 ('abroad', 1),
 ('abrupt', 1),
 ('abruptly', 5),
 ('absolved', 1),
 ('abuse', 2),
 ('abuses', 1),
 ('accidentally', 1),
 ('accompanied', 1),
 ('according', 1),
 ('accusation', 1),
 ('accustomed', 2),
 ('ache', 1),
 ('aches', 1),
 ('achingly', 2),
 ('acoustic', 1),
 ('across', 24),
 ('actually', 1),
 ('adagio', 1),
 ('adapted', 1),
 ('address', 1),
 ('addressed', 1),
 ('addresses', 1),
 ('adept', 1),
 ('adieu', 4),
 ('adjacent', 1),
 ('adjoining', 1),
 ('adjust', 1),
 ('admired', 1),
 ('ado', 1),
 ('advanced', 1),
 ('advances', 3),
 ('adversaries', 1),
 ('aerial', 5),
 ('affection', 1),
 ('afford', 2),
 ('african', 1),
 ('after', 7),
 ('afternoon', 42)]

## STREAMING WITH PySPARK

In [4]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)

23/10/28 11:32:54 WARN Utils: Your hostname, josarthur-HP-ProBook-450-G8-Notebook-PC resolves to a loopback address: 127.0.1.1; using 10.3.7.83 instead (on interface enp2s0)
23/10/28 11:32:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/28 11:32:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# Create a DStream that will connect to hostname:port, like localhost:9999
# Firewalls might block this!
lines = ssc.socketTextStream("localhost", 9999)

In [6]:
# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

In [7]:
# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [9]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

23/10/28 11:36:29 WARN StreamingContext: StreamingContext has already been started


-------------------------------------------
Time: 2023-10-28 11:36:30
-------------------------------------------



-------------------------------------------
Time: 2023-10-28 11:36:31
-------------------------------------------



-------------------------------------------
Time: 2023-10-28 11:36:32
-------------------------------------------



-------------------------------------------
Time: 2023-10-28 11:36:33
-------------------------------------------



...

-------------------------------------------
Time: 2023-10-28 11:38:22
-------------------------------------------



-------------------------------------------
Time: 2023-10-28 11:38:23
-------------------------------------------



-------------------------------------------
Time: 2023-10-28 11:38:24
-------------------------------------------



-------------------------------------------
Time: 2023-10-28 11:38:25
-------------------------------------------



KeyboardInterrupt: 

## Worlcount on cluster with master slave 



### Exécuter spark En local :  $ spark-submit --master local[*] wordcount.py test100.txt



### En mode cluster Lancer le master: $ start-master.sh --host 192.168.43.2 --port 7077



### Lancer les workers exécuter sur chaque worker : $ start-slave.sh spark://192.168.43.2:7077

### Pour Arreter le master : $ stop-master.sh --host 192.168.43.2 --port 7077